In [36]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
 
import scipy.io as sio
 
from torchvision import datasets, transforms 
from torch.utils.data import DataLoader
import torchvision.models as models 
import torchvision.utils as utils  

import numpy as np
import cv2 
import pyrealsense2 as rs

In [37]:
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [38]:
from torchvision.models import resnet50,ResNet50_Weights
net=resnet50(weights=ResNet50_Weights.IMAGENET1K_V2).to(device)

### strip the last layer
feature_extractor = torch.nn.Sequential(*list(net.children())[:-1])

In [48]:
class BC(nn.Module):
    def __init__(self, input_size=2048, output_size=6):
        super(BC, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x=self.fc2(x)
        return x

In [49]:
model =  BC().to(device)
model.load_state_dict(torch.load('resnet50sawyer_pick_bc_xyz.pt'))

<All keys matched successfully>

In [31]:
class MyRealSense:
    def __init__(self):
        self.pipe = rs.pipeline()
        self.profile = self.pipe.start()

    def get_current_frame(self, scale=0.5):
        frames = self.pipe.wait_for_frames()
        color_frame = frames.get_color_frame() 

        image=np.asanyarray(color_frame.get_data())
        image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        image = cv2.resize(image, (int(image.shape[1]*scale), int(image.shape[0]*scale) ) , interpolation = cv2.INTER_AREA)
        return image 

    def close(self):
        self.pipe.stop()

In [64]:
cam=MyRealSense()

while True:
    image = cam.get_current_frame()
    x=torch.tensor(image.transpose(2,0,1)).float().to(device)[None]
    fs=feature_extractor(x).squeeze()
    paction=model(fs).detach()

    cv2.imshow("press q to close", image)
    key = cv2.waitKey(1)
    # Press esc or 'q' to close the image window
    if key & 0xFF == ord('q') or key == 27:
        cv2.destroyAllWindows()
        cam.close()
        break

In [34]:
# cv2.destroyAllWindows()
# cam.close()

(360, 640, 3)

In [71]:
paction.cpu().numpy()

array([ 0.45156348,  0.22939338,  0.2790616 , -1.3480388 , -0.00167866,
        0.73156327], dtype=float32)

In [72]:
pa=paction.cpu().numpy()
str(pa)

'[ 0.45156348  0.22939338  0.2790616  -1.3480388  -0.00167866  0.73156327]'

In [74]:
pa

array([ 0.45156348,  0.22939338,  0.2790616 , -1.3480388 , -0.00167866,
        0.73156327], dtype=float32)

In [61]:

paction.shape

torch.Size([6])

In [75]:
list(pa)

[0.45156348, 0.22939338, 0.2790616, -1.3480388, -0.0016786605, 0.73156327]

In [68]:
traced_model = torch.jit.trace(model.forward,fs)

/home/ns/miniconda3/lib/python3.10/site-packages/torch/jit/_trace.py:154: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/build/aten/src/ATen/core/TensorBody.h:486.)
  if a.grad is not None:


In [69]:
traced_model.save("traced_bc.pt")

In [141]:
i=1
fn=f"sawyer_pick_{i}_imgs_xyz.mat"
data=sio.loadmat(fn)
actions=data['xyz']
imgs=data['imgs']

In [142]:
imgs.shape, actions.shape

((102, 540, 960, 3), (102, 6))

In [143]:
for i in range(len(imgs)):
    img=imgs[i]
    action=actions[i]

In [144]:
img.shape, action.shape

((540, 960, 3), (6,))